In [1]:
import pandas as pd
import numpy as np

url = 'https://raw.githubusercontent.com/krishnaik06/Recommendation_complete_tutorial/master/KNN%20Movie%20Recommendation/movies.csv'
movies = pd.read_csv(url,sep=',')

In [2]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
url = 'https://raw.githubusercontent.com/krishnaik06/Recommendation_complete_tutorial/master/KNN%20Movie%20Recommendation/ratings.csv'
ratings = pd.read_csv(url,sep=',')


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
df=pd.merge(ratings, movies,on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [6]:
df=df.drop(columns=['genres','timestamp'])

In [7]:
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [12]:
movie_ratingCount = (df.
     groupby(by = ['title'])['rating'].
     count().reset_index().
                    rename(columns={'rating':'totalRatingCount'})
                    [['title','totalRatingCount']])
movie_ratingCount.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [13]:
# Combine the ratingCount into df
rating_with_totalratingCount=pd.merge(df,movie_ratingCount,on='title')
rating_with_totalratingCount.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [18]:
movie_ratingCount.describe()

,totalRatingCount
count,9719.000
mean,10.375
std,22.406
min,1.000
25%,1.000
50%,3.000
75%,9.000
max,329.000


In [20]:
# Delete the movies rated for 50 times and less
rating_pop_movies=rating_with_totalratingCount[rating_with_totalratingCount['totalRatingCount']>50].sort_values('totalRatingCount',ascending=False)


In [23]:
rating_pop_movies.head()

,userId,movieId,rating,title,totalRatingCount
2534,199,356,2.000,Forrest Gump (1994),329
2545,221,356,3.500,Forrest Gump (1994),329
2538,205,356,4.000,Forrest Gump (1994),329
2539,209,356,4.500,Forrest Gump (1994),329
2540,211,356,4.500,Forrest Gump (1994),329


In [26]:
# Create a pivot table
moviemat=rating_pop_movies.pivot_table(index='title',columns='userId',values='rating').fillna(0)
moviemat.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,3.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000
12 Angry Men (1957),0.000,0.000,0.000,5.000,0.000,0.000,0.000,0.000,0.000,0.000,...,5.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2001: A Space Odyssey (1968),0.000,0.000,0.000,0.000,0.000,0.000,4.000,0.000,0.000,0.000,...,0.000,0.000,5.000,0.000,0.000,5.000,0.000,3.000,0.000,4.500
28 Days Later (2002),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.500,0.000,5.000
300 (2007),0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,3.000,...,0.000,0.000,0.000,0.000,3.000,0.000,0.000,5.000,0.000,4.000


In [31]:
# Convert the pivot table to a sparse matrix
from scipy.sparse import csr_matrix
moviematrix=csr_matrix(moviemat.values)
moviematrix.toarray()

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 3. , 0. , 4.5],
       ...,
       [5. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 3. , 0. , ..., 0. , 0. , 3.5],
       [0. , 0. , 0. , ..., 3. , 0. , 4. ]])

In [32]:
# Using KNN to find the cosine similarity of the movies
from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(moviematrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [34]:
moviematrix.shape

(437, 606)

In [65]:
query_index=np.random.choice(moviematrix.shape[0])
print(query_index)

58


In [66]:
distances,indices=model_knn.kneighbors(moviemat.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)

In [67]:
len(distances.flatten())

6

In [68]:
indices

array([[ 58,   2,  14, 386, 272, 377]])

In [69]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(moviemat.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, moviemat.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Blade Runner (1982):

1: 2001: A Space Odyssey (1968), with distance of 0.3292643457846739:
2: Alien (1979), with distance of 0.3924053698047032:
3: Terminator, The (1984), with distance of 0.411775935019875:
4: Monty Python and the Holy Grail (1975), with distance of 0.4162335116953314:
5: Star Wars: Episode V - The Empire Strikes Back (1980), with distance of 0.41755051089362094:
